In [5]:
#40
class Morph(object):   
    def __init__(self, line):
        self.surface = line[0]
        self.base = line[7]
        self.pos = line[1]
        self.pos1 = line[2]

import re
        
def create_morph(fname):    
    with open(fname, "r", encoding="utf-8") as f:
        text = f.read().splitlines()
        sentense = [re.split("[\t, ]", t) for t in text]
        morphs = []
        temp = [] 
        for word in sentense:
            if word[0] == "*" and len(word) == 5:
                continue
            elif word[0] != "EOS":
                temp.append(Morph(word))
            else:
                morphs.append(temp)
                temp = []
                                
    return morphs
morph3 = create_morph('neko.txt.cabocha')[2]
for morph in morph3:
    print("{}\t{}\t{}\t{}".format(morph.surface, 
                                    morph.base, 
                                    morph.pos, 
                                    morph.pos1))

　	　	記号	空白
吾輩	吾輩	名詞	代名詞
は	は	助詞	係助詞
猫	猫	名詞	一般
で	だ	助動詞	*
ある	ある	助動詞	*
。	。	記号	句点


In [107]:
#41
class Chunk(object):
    def __init__(self, line):
        self.morphs = []
        self.dst = int(line[2].strip("D"))
        self.srcs = []
            
    def join_surface(self):
        return "".join([morph.surface for morph in self.morphs])
     
    #42
    def join_surface_without_mark(self):
        return "".join([morph.surface for morph in self.morphs
                       if morph.pos != "記号"])

    def bool_pos_in(self, pos):
        for morph in self.morphs:
            if morph.pos == pos:
                return True
        return False
    #49
    def mask(self, mask):
        result = ""
        for morph in self.morphs:
            if morph.pos != "記号":
                if morph.pos == "名詞":
                    result += mask
                    mask = ""  
                else:
                    result += morph.surface   
        return result 
    
def create_chunk(fname):    
    with open(fname, "r", encoding="utf-8") as f:
        text = f.read().splitlines()
        sentense = [re.split("[\t, ]", t) for t in text]
        chunks = []
        temp = []
        for line in sentense:
            if line[0] == "*" and len(line) == 5:
                chunk = Chunk(line)
                temp.append(chunk)
            elif line[0] != "EOS":
                chunk.morphs.append(Morph(line))
            else:
                for i, chunk in enumerate(temp):
                    if chunk.dst != -1:
                        temp[chunk.dst].srcs.append(i)  
                chunks.append(temp)
                temp = []
        
    return chunks

In [7]:
chunk7 = create_chunk('neko.txt.cabocha')[7]
for item in chunk7:
    print("{}\t{}\t{}".format(item.join_surface(), 
                              item.dst, 
                              item.srcs))

吾輩は	5	[]
ここで	2	[]
始めて	3	[1]
人間という	4	[2]
ものを	5	[3]
見た。	-1	[0, 4]


In [9]:
chunks_sentences = create_chunk("neko.txt.cabocha")[:7]

for chunks in chunks_sentences:
    for chunk in chunks:
        if chunk.dst != -1:
            s = chunk.join_surface_without_mark()
            d = chunks[chunk.dst].join_surface_without_mark()
            if s != "" and d != "":
                print("{}\t{}".format(s, d))

吾輩は	猫である
名前は	無い
まだ	無い
どこで	生れたか
生れたか	つかぬ
とんと	つかぬ
見当が	つかぬ
何でも	薄暗い
薄暗い	所で
じめじめした	所で
所で	泣いて
ニャーニャー	泣いて
泣いて	記憶している
いた事だけは	記憶している


In [10]:
chunks_sentences = create_chunk("neko.txt.cabocha")[:7]

for chunks in chunks_sentences:
    for chunk in chunks:
        if chunk.bool_pos_in("名詞") and chunks[chunk.dst].bool_pos_in("動詞"):
            if chunk.dst != -1:
                s = chunk.join_surface_without_mark()
                d = chunks[chunk.dst].join_surface_without_mark()
                if s != "" and d != "":
                    print("{}\t{}".format(s, d))

どこで	生れたか
見当が	つかぬ
所で	泣いて
ニャーニャー	泣いて
いた事だけは	記憶している


In [90]:
chunk_sentences = create_chunk("neko.txt.cabocha")[6]
import pydot
edges = []
for chunks in chunks_sentences:
        if chunk.dst != -1:
            edges.append((chunk.join_surface_without_mark().encode('utf-8'),chunks[chunk.dst].join_surface_without_mark().encode('utf-8')))
g = pydot.graph_from_edges(edges,directed = True)
g.write_jpeg('dot.jpg', prog='dot')
            

In [91]:
from PIL import Image
Image.open('dot.jpg').show()

In [17]:
chunks_sentences = create_chunk("neko.txt.cabocha")[:7]
for chunks in chunks_sentences:
    for chunk in chunks:
        print(chunk.surface)

AttributeError: 'Chunk' object has no attribute 'surface'

In [31]:
#45
chunks_sentences = create_chunk("neko.txt.cabocha")[:7]

kaku_all = []
for chunks in chunks_sentences:
    for chunk in chunks:
                            
        verb_lst = []
        if chunk.bool_pos_in("動詞"):
            for morph in chunk.morphs:
                if morph.pos == '動詞':
                    verb_lst.append(morph.base)
                else:
                    continue
        joshi_lst = set()
        for i in chunk.srcs:
            if chunks[i].bool_pos_in("助詞"):
                for morph in chunks[i].morphs:
                    if morph.pos == "助詞":
                            joshi_lst.add(morph.surface)
            else:
                    continue
        if (joshi_lst != set()) and (verb_lst != []):
            kaku_tmp = []
            kaku_tmp.append(verb_lst[0])
            kaku_tmp.extend(sorted(joshi_lst))
            kaku_all.append(kaku_tmp)

kaku_all

[['生れる', 'で'], ['つく', 'か', 'が'], ['泣く', 'で'], ['する', 'だけ', 'て', 'は']]

In [49]:
#46
chunks_sentences = create_chunk("neko.txt.cabocha")[:7]

kaku_frame_all = []
for chunks in chunks_sentences:
    for chunk in chunks:
        verb_lst = []
        if chunk.bool_pos_in("動詞"):
            for morph in chunk.morphs:
                if morph.pos == '動詞':
                    verb_lst.append(morph.base)
                else:
                    continue
        joshi_lst = []
        for i in chunk.srcs:
            if chunks[i].bool_pos_in("助詞"):
                for morph in chunks[i].morphs:
                    if morph.pos == "助詞":
                        joshi_tmp = []
                        joshi_tmp.append(morph.surface)
                        joshi_tmp.append(chunks[i].join_surface_without_mark())
                        joshi_lst.append(joshi_tmp)
            else:
                    continue
        if (joshi_lst != []) and (verb_lst != []):
            kaku_tmp = []
            kaku_tmp.append(verb_lst[0])
            for i in range(len(joshi_lst)):
                aaa = set()
                aaa.add(joshi_lst[i][0])
                kaku_tmp.extend(sorted(aaa))
            for j in range(len(joshi_lst)):
                kaku_tmp.extend(sorted(joshi_lst[j][1:]))
            kaku_frame_all.append(kaku_tmp)
kaku_frame_all

[['生れる', 'で', 'どこで'],
 ['つく', 'か', 'が', '生れたか', '見当が'],
 ['泣く', 'で', '所で'],
 ['する', 'て', 'だけ', 'は', '泣いて', 'いた事だけは', 'いた事だけは']]

In [61]:
#47
chunks_sentences = create_chunk("neko.txt.cabocha")[:100]

kino_verb_all = []
for chunks in chunks_sentences:
    for chunk in chunks:
        verb_lst = []
        if chunk.bool_pos_in("動詞"):
            for morph in chunk.morphs:
                if morph.pos == '動詞':
                    verb_lst.append(morph.base)
                else:
                    continue
        sahen_etc=''
        joshi_lst = []
        for i in chunk.srcs:
            if chunks[i].bool_pos_in("助詞"):
                for j in range(len(chunks[i].morphs)-1):
                        if chunks[i].morphs[j].pos == "名詞" \
                        and chunks[i].morphs[j].pos1 == "サ変接続" \
                        and chunks[i].morphs[j+1].pos == "助詞" \
                        and chunks[i].morphs[j+1].surface == "を":
                            sahen_etc = chunks[i].morphs[j].surface + chunks[i].morphs[j+1].surface
                        else:
                            continue
                for morph in chunks[i].morphs:
                    if morph.pos == "助詞" and chunks[i].join_surface_without_mark() != sahen_etc:
                        joshi_tmp = []
                        joshi_tmp.append(morph.surface)
                        joshi_tmp.append(chunks[i].join_surface_without_mark())
                        joshi_lst.append(joshi_tmp)
            else:
                    continue
        if (joshi_lst != []) and (verb_lst != []) and ( sahen_etc !=''):
            kaku_tmp = []
            kaku_tmp.append(sahen_etc+verb_lst[0])
            for i in range(len(joshi_lst)):
                aaa = set()
                aaa.add(joshi_lst[i][0])
                kaku_tmp.extend(sorted(aaa))
            for j in range(len(joshi_lst)):
                kaku_tmp.extend(sorted(joshi_lst[j][1:]))
            kino_verb_all.append(kaku_tmp)
kino_verb_all

[['決心をする', 'と', 'こうと'], ['返報をする', 'んで', '偸んで'], ['昼寝をする', 'が', '彼が']]

In [67]:
#48
chunks_sentences = create_chunk("neko.txt.cabocha")[:100]

from_norm_path = []
for chunks in chunks_sentences:
    for chunk in chunks:
        if chunk.bool_pos_in("名詞"):
            path_tmp=[]
            path_tmp.append(chunk.join_surface_without_mark())
            i = chunk.dst
            if i == -1:
                continue
            while i != -1:
                path_tmp.append(chunks[i].join_surface_without_mark())
                i = chunks[i].dst
            from_norm_path.append(path_tmp)
            
result = [' -> '.join(path) for path in from_norm_path]            
result                

['吾輩は -> 猫である',
 '名前は -> 無い',
 'どこで -> 生れたか -> つかぬ',
 '見当が -> つかぬ',
 '何でも -> 薄暗い -> 所で -> 泣いて -> 記憶している',
 '所で -> 泣いて -> 記憶している',
 'ニャーニャー -> 泣いて -> 記憶している',
 'いた事だけは -> 記憶している',
 '吾輩は -> 見た',
 'ここで -> 始めて -> 人間という -> ものを -> 見た',
 '人間という -> ものを -> 見た',
 'ものを -> 見た',
 'あとで -> 聞くと -> 種族であったそうだ',
 'それは -> 種族であったそうだ',
 '書生という -> 人間中で -> 種族であったそうだ',
 '人間中で -> 種族であったそうだ',
 '一番 -> 獰悪な -> 種族であったそうだ',
 '獰悪な -> 種族であったそうだ',
 '書生というのは -> 話である',
 '我々を -> 捕えて -> 煮て -> 食うという -> 話である',
 '当時は -> なかったから -> 思わなかった',
 '何という -> 考も -> なかったから -> 思わなかった',
 '考も -> なかったから -> 思わなかった',
 '彼の -> 掌に -> 載せられて -> 持ち上げられた -> 時 -> フワフワした -> 感じが -> あったばかりである',
 '掌に -> 載せられて -> 持ち上げられた -> 時 -> フワフワした -> 感じが -> あったばかりである',
 'スーと -> 持ち上げられた -> 時 -> フワフワした -> 感じが -> あったばかりである',
 '時 -> フワフワした -> 感じが -> あったばかりである',
 '感じが -> あったばかりである',
 '掌の -> 上で -> 落ちついて -> 見たのが -> 人間という -> ものの -> 見始であろう',
 '上で -> 落ちついて -> 見たのが -> 人間という -> ものの -> 見始であろう',
 '書生の -> 顔を -> 見たのが -> 人間という -> ものの -> 見始であろう',
 '顔を -> 見たのが -> 人間という -> ものの -> 見始であろう',

In [139]:
#49
chunks_sentences = create_chunk("neko.txt.cabocha")[:10]

import copy
result=[]
for chunks in chunks_sentences:
    path_lst=[]
    path_masked_lst= []
    for chunk in chunks:
        if chunk.bool_pos_in("名詞"):
            path_tmp=[]
            path_tmp_masked =[]
            # 最初の名詞句を保存
            path_tmp.append(chunk.join_surface_without_mark())
            # 最初の名詞句をXに変換し保存
            path_tmp_masked.append(chunk.mask('X'))
            # かかり先のindex
            i = chunk.dst
            # 文末に到達するまでループ
            if i == -1:
                continue
            while i != -1:
                # 文節ごとに結合
                path_tmp.append(chunks[i].join_surface_without_mark())
                path_tmp_masked.append(chunks[i].join_surface_without_mark())
                # 1本道の場合
                if chunks[i].bool_pos_in("名詞"):
                    # 二つ目の名詞句が存在すれば, XをYに変換して保存
                    path_tmp_copied = copy.copy(path_tmp_masked)
                    path_tmp_copied[-1] = chunks[i].mask('Y')
                    direct_path = ' -> '.join(path_tmp_copied)
                    result.append(direct_path)
                    i = chunks[i].dst
                else:
                    i = chunks[i].dst
            for i in range(len(path_lst)):
                # 二つの文末が一致した場合
                if path_lst[i][-1] == path_tmp[-1]:
                    #YがXに独立して存在するとき
                    if [item for item in  path_tmp if item not in path_lst[i]] !=[]:
                        uuu_path = []
                        # maskしたもの
                        path_tmp_masked_copied= copy.copy(path_tmp_masked)
                        path_tmp_masked_copied[0] = path_tmp_masked_copied[0].replace('X', 'Y')
                        X_only_paths = [item for item in path_masked_lst[i] if item not in path_tmp_masked_copied]
                        Y_only_paths = [item for item in  path_tmp_masked_copied if item not in path_masked_lst[i]]
                        common_paths = [item for item in  path_tmp_masked_copied if item  in path_masked_lst[i]]
                        X_path = ' -> '.join(X_only_paths)
                        Y_path = ' -> '.join(Y_only_paths)
                        common_path = ' -> '.join(common_paths)
                        uuu_path.append(X_path)
                        uuu_path.append(Y_path)
                        uuu_path.append(common_path)
                        indirect_path = ' | '.join(uuu_path)
                        result.append(indirect_path)
                        uuu_path= []
            # 文末まで到達したものを保存
            path_lst.append(path_tmp)
            path_masked_lst.append(path_tmp_masked)
result

['Xは -> Yである',
 'Xで -> 生れたか | Yが | つかぬ',
 'Xでも -> 薄暗い -> Yで',
 'Xでも -> 薄暗い -> 所で -> 泣いて -> Yしている',
 'Xで -> 泣いて -> Yしている',
 'X -> 泣いて -> Yしている',
 'Xでも -> 薄暗い -> 所で | Y | 泣いて -> 記憶している',
 'Xで | Y | 泣いて -> 記憶している',
 'Xだけは -> Yしている',
 'Xでも -> 薄暗い -> 所で -> 泣いて | Yだけは | 記憶している',
 'Xで -> 泣いて | Yだけは | 記憶している',
 'X -> 泣いて | Yだけは | 記憶している',
 'Xで -> 始めて -> Yという',
 'Xで -> 始めて -> 人間という -> Yを',
 'Xは | Yで -> 始めて -> 人間という -> ものを | 見た',
 'Xという -> Yを',
 'Xは | Yという -> ものを | 見た',
 'Xは | Yを | 見た',
 'Xで -> 聞くと -> Yであっただ',
 'Xは -> Yであっただ',
 'Xで -> 聞くと | Yは | 種族であったそうだ',
 'Xという -> Yで',
 'Xという -> 人間中で -> Yであっただ',
 'Xで -> 聞くと | Yという -> 人間中で | 種族であったそうだ',
 'Xは | Yという -> 人間中で | 種族であったそうだ',
 'Xで -> Yであっただ',
 'Xで -> 聞くと | Yで | 種族であったそうだ',
 'Xは | Yで | 種族であったそうだ',
 'X -> Yな',
 'X -> 獰悪な -> Yであっただ',
 'Xで -> 聞くと | Y -> 獰悪な | 種族であったそうだ',
 'Xは | Y -> 獰悪な | 種族であったそうだ',
 'Xという -> 人間中で | Y -> 獰悪な | 種族であったそうだ',
 'Xで | Y -> 獰悪な | 種族であったそうだ',
 'Xな -> Yであっただ',
 'Xで -> 聞くと | Yな | 種族であったそうだ',
 'Xは | Yな | 種族であったそうだ',
 'Xという -> 人間中